In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, precision_recall_curve

from scripts import featurizer_xgb

import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
df = pd.read_csv("../data/dfff.csv", index_col=False)

In [13]:
df.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,R_28,D_139,D_140,D_144,D_145,customer_ID,S_2,D_63,D_64,target
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,NaN,0.001613,0.029088,...,0.000518,0.003183,0.005264,0.004306,0.002719,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-08-10,CO,O,0
1,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,0.001186,1.000928,0.005655,0.007953,0.095926,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-07-04,CO,O,0
2,0.392041,0.005770,0.029682,0.810262,0.006672,0.180378,0.005028,NaN,0.126794,0.210560,...,0.006967,NaN,NaN,NaN,NaN,4488928018c20bf6261e69c8a94a47b6206a2fe91480b8...,2017-07-29,CR,NaN,1
3,0.673978,0.588604,0.295201,0.024650,0.007951,0.125583,0.008760,NaN,0.005867,0.175950,...,0.000805,0.003558,0.008268,0.003020,0.002798,9296e983c3e4f99b046952397fc4ba950abf1032a9bd6c...,2017-03-30,CO,R,1
4,0.778075,0.006930,0.056721,0.819369,0.004922,0.089214,0.006884,0.041515,0.002841,0.001179,...,0.009053,0.004670,0.000125,0.000819,0.000342,3fa10314686e0e51d8f76235eceab91d70aa3f24a98c3e...,2018-01-17,CL,O,0


In [14]:
d_feats = [c for c in df.columns if c.startswith('D_')]
s_feats = [c for c in df.columns if c.startswith('S_')]
p_feats = [c for c in df.columns if c.startswith('P_')]
b_feats = [c for c in df.columns if c.startswith('B_')]
r_feats = [c for c in df.columns if c.startswith('R_')]

In [16]:
#d_feats = [c for c in df.columns if c.startswith('D_')]
#s_feats = [c for c in df.columns if c.startswith('S_')]
#p_feats = [c for c in df.columns if c.startswith('P_')]
#b_feats = [c for c in df.columns if c.startswith('B_')]
#r_feats = [c for c in df.columns if c.startswith('R_')]

#type_feats = [d_feats, s_feats, p_feats, b_feats, r_feats]
#type_feats_name = ["d_feats","s_feats","p_feats","b_feats","r_feats"]
#i = 0

#for type_col in type_feats:
#    df[type_feats_name[i] + "_mean"] = df[type_col].mean(axis = 1, skipna = True, numeric_only=True)
#    i = i + 1
#df.head()
    

In [17]:
df_featurizado = featurizer_xgb.featurizer(df)

-------


In [18]:
df_featurizado

,target,Year,Month,Day,customer_id_repeat_size,d_feats_mean,s_feats_mean,p_feats_mean,b_feats_mean,r_feats_mean
0,0,2017,8,10,2,0.352906,0.141439,0.501111,0.127543,0.091620
1,0,2017,10,1,2,0.362819,0.246337,0.463509,0.140606,0.091351
2,0,2017,7,4,3,0.339439,-0.026898,0.448435,0.126645,0.096931
3,0,2017,6,20,3,0.354992,0.130245,0.447317,0.126185,0.096216
4,0,2017,9,20,3,0.321710,0.195712,0.473658,0.114336,0.096874
...,...,...,...,...,...,...,...,...,...,...
276567,1,2017,7,18,1,0.427684,0.212867,0.356782,0.265898,0.243918
276568,0,2017,9,6,1,0.188680,0.232531,0.743498,0.143550,0.091943
276569,1,2018,1,16,1,0.224079,0.193630,0.365820,0.256912,0.123065
276570,0,2017,7,18,1,0.356399,0.214661,0.343914,0.235310,0.090861


In [19]:
df_featurizado.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276572 entries, 0 to 276571
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   target                   276572 non-null  int64  
 1   Year                     276572 non-null  int64  
 2   Month                    276572 non-null  int64  
 3   Day                      276572 non-null  int64  
 4   customer_id_repeat_size  276572 non-null  int64  
 5   d_feats_mean             276572 non-null  float64
 6   s_feats_mean             276572 non-null  float64
 7   p_feats_mean             276572 non-null  float64
 8   b_feats_mean             276572 non-null  float64
 9   r_feats_mean             276572 non-null  float64
dtypes: float64(5), int64(5)
memory usage: 23.2 MB
